In [1]:
from glob import glob
import os
import pandas as pd
from pathlib import Path
import re
import numpy as np

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

import mmcv
from mmcv import Config
cfg = Config.fromfile('/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/cascade_rcnn_lagfart.py')

1.12.1 True
2.25.2
10.2
GCC 7.3


In [2]:
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
checkpoint = '/home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/latest.pth'
model = init_detector(cfg, checkpoint, device='cuda:0')

load checkpoint from local path: /home/erik/Riksarkivet/Projects/lagfarter/models/det_crcnn_iter_2/latest.pth


In [ ]:
#läs in bildbatcherna
#för vaje bild
#kör modellen
#om == 1 property, och conf > 0.95:
#Hämta ut gt för bilden (bara property name)
#skriv ut filnamn|bounding box|gt

In [3]:
df_lagf = pd.read_csv('/home/erik/Riksarkivet/Projects/lagfarter/data/db_files/Uttag_Lagfartsbok_2022-09-14', sep='\t', encoding='utf8', dtype=str)

In [21]:
paranthesis = re.compile('\(.+\)')
socken = re.compile('.+-')
litt = re.compile('\s[Ll]itt')
litt_char = re.compile('\s[a-zA-Z]{1}\s') #doesn't work, split up on two
last_litt = re.compile('\s[a-zA-Z]{1}$')
first = re.sub(litt, '', 'malmö A B')
first = re.sub(litt_char, ' ', first)
first = re.sub(last_litt, '', first)
print(first)

malmö


In [4]:
#rewrite as a function with field as parameter, clean fastnamn as a function, otherwise, everything works, move hd to gpu start running

imgs = glob(os.path.join('/media/erik/T7/Data/Lagfarter/cropped_imgs_ind_batches', '*', '**'))
imgs.sort()

ignore = ['Försättsblad', 'Slutskylt', 'Pärm', 'Information', 'Blankt uppslag']

with open(os.path.join('/home/erik/Riksarkivet/Projects/lagfarter/data/create_htr_set_test', 'gt_all_2.txt'), 'w') as f:

    for i, img_p in enumerate(imgs):
        
        try:
        
            batch_filename = '/'.join(img_p.split('/')[-2:])

            arkis_nr = Path(img_p).stem.split('_')[1]
            series = Path(img_p).stem.split('_')[2]
            volume = Path(img_p).stem.split('_')[3]
            page_nr = str(int(Path(img_p).stem.split('_')[4]))
        
        
            row = df_lagf[(df_lagf['ARKISNR'] == arkis_nr) & (df_lagf['ARKISSIG'] == series) & (df_lagf['ARKISVOL'] == volume) & (df_lagf['APAGE'] == page_nr)]
            gt = str(row.iloc[0]['FAST'])
            gt = gt.strip()

            if gt in ignore or 'Register' in gt:
                continue

            paranthesis_re = re.compile('\(.+\)')
            socken_re = re.compile('.+-')
            litt_re = re.compile('\s[Ll]itt')
            litt_char_re = re.compile('\s[a-zA-Z]{1}\s')
            last_litt_re = re.compile('\s[a-zA-Z]{1}$')

            gt = re.sub(paranthesis_re, '', gt)
            gt = re.sub(socken_re, '', gt)
            gt = re.sub(litt_re, '', gt)
            gt = re.sub(litt_char_re, ' ', gt)
            gt = re.sub(last_litt_re, '', gt)
            gt = gt.strip()

            if gt == '':
                continue

            img = mmcv.imread(img_p)
            result = inference_detector(model, img)
            
            fast_names_result = result[1]
            fast_names_result = [x for x in fast_names_result if x[4] > 0.95]
            
            if len(fast_names_result) == 1:
                x1 = int(fast_names_result[0][0])
                y1 = int(fast_names_result[0][1])
                x2 = int(fast_names_result[0][2])
                y2 = int(fast_names_result[0][3])

                #bbox = np.array([x1,y1,x2,y2])
                #img_crop = mmcv.imcrop(img, bbox)

                #file_out = os.path.join('/home/erik/Riksarkivet/Projects/lagfarter/data/create_htr_set_test_crop', batch_filename.replace('/', '_'))

                #mmcv.imwrite(img_crop, file_out)

                f.write(batch_filename + '|' + str(x1) + '|' + str(y1) + '|' + str(x2) + '|' + str(y1) + '|' + str(x2) + '|' + str(y2) + '|' + str(x1) + '|' + str(y2) + '|' + gt + '\n')

            if i % 1000 == 0:
                print(i)

        except Exception as e:
            print(e)
            continue
        


single positional indexer is out-of-bounds


/home/erik/Riksarkivet/Projects/lagfarter/mmdetection/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
3000
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
4000
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
5000
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
6000
single positional indexer is out-of-bounds
single positional indexer is out-of-bounds
single positional indexer is out-o